In [ ]:
import numpy as np
from scipy.integrate import quad, tplquad
import vtk
import pyvista as pv

from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve

import matplotlib.pyplot as plt


---

# Problem description

The goal is to solve for the displacement field of a linear elastic solid under a given load. 
Instead of employing the Garlekin method of weighted residuals, we will use this time the hamiltonian principle of stationary action: 

The action functional is given by:
$$
S = \int_{t_0}^{t_f} L(q, \dot{q}) dt
$$

Where $L$ is the Lagrangian of the system, which is given by: $L = T- V$. Where $V$ is the potential energy of the system and $T$ is the kinetic energy of the system. $q$ is the generalized coordinate of the system and $\dot{q}$ is the generalized velocity of the system.

The stationary action principle states that the true motion of the system is such that the action is stationary, i.e. the variation of the action is zero:

$$
\delta S = 0
$$

By applying the calculus of variations to $\delta S$, the equations of motion for the system are derived. This leads to the Euler-Lagrange equations:

$$
\frac{d}{dt} \left( \frac{\partial L}{\partial \dot{q}} \right) - \frac{\partial L}{\partial q} = 0
$$

The first step of this approach is to derive the Lagrangian of the system. 

---

<br>
<br>
<br>
<br>
<br>

---

## State variables interpolation in the mesh elements

The displacement field as well as the stress and strain fields are linearly interpolated in the mesh elements (tetrahedra in this case) based on the values at the element nodes. The position inside an element can be defined based on the position of the element nodes and the barycentric coordinates of the point inside the element. This gives 
the following relationship: 

$$
X = \phi_0(X) X_0 + \phi_1(X) X_1 + \phi_2(X) X_2 + \phi_3(X) X_3
$$

Here $X$ denotes the position of a point in the reference/underformed geometry. The position of the same point in the deformed geometry is denoted by $x(X)$. The functions $\phi_i(X)$ are the barycentric coordinates of the point $X$ inside the element. This interpolation given above is also valid for any function such as the position in the deformed geometry $x(X)$, the displacement field $u(X)$, the strain field $\varepsilon(X)$, the stress field $\sigma(X)$, etc. 


We can rewrite the above equation into a matrix vector form: 

$$
X = 
\begin{bmatrix}
 \mid& \mid&  \mid& \mid  \\
 X_0&  X_1&  X_2&  X_3 \\
 \mid&  \mid&  \mid& \mid \\
\end{bmatrix}  

\begin{bmatrix}
\phi_0(X)\\
\phi_1(X)\\
\phi_2(X)\\
\phi_3(X)\\
\end{bmatrix}
$$

This system is not solvable, but with the fact that the sum of the barycentric coordinates is always equal to 1, we can rewrite the above equation as:

$$
X - X_0 = 
\underbrace{
\begin{bmatrix}
 \mid& \mid&  \mid \\
  X_1 - X_0&  X_2- X_0&  X_3- X_0 \\
 \mid&  \mid&  \mid&\\
\end{bmatrix}}_{=\;T}

\begin{bmatrix}
\phi_1(X)\\
\phi_2(X)\\
\phi_3(X)\\
\end{bmatrix}
$$

and 

$$
\phi_0(X) = 1 - \phi_1(X) - \phi_2(X) - \phi_3(X)
$$


we can therefore write that

$$
\begin{bmatrix}
\phi_1(X)\\
\phi_2(X)\\
\phi_3(X)\\
\end{bmatrix} = 

T^{-1} (X - X_0)
$$

and 

$$
\phi_0(X) = 1 - \vec{1}^T T^{-1} (X - X_0)
$$

where $\vec{1}$ is a vector of ones.

Assembling everything together, we obtain the following expression for the barycetric position of a point inside an element:


$$ 
\begin{bmatrix}
\phi_0(X)\\
\phi_1(X)\\
\phi_2(X)\\
\phi_3(X)\\
\end{bmatrix} =  


\begin{bmatrix}
1\\
0\\
0\\
0\\
\end{bmatrix} 
+
\underbrace{
\begin{bmatrix}
& & & -\vec{1}^{T} \; T^{-1} & & & \\
-&-&-&-&-&-& \\
& & & & & & \\
& & &T^{-1}& & & \\
& & & & & & \\
\end{bmatrix}}_{=D \;(4 \times 3)}


\underbrace{
(X - X_0)}_{(3 \times 1)}


$$

The position in the deformed geometry is given by:

$$
x(X) = \underbrace{
\begin{bmatrix}
 \mid& \mid&  \mid& \mid  \\
 x_0&  x_1&  x_2&  x_3 \\
 \mid&  \mid&  \mid& \mid \\
\end{bmatrix}}_{(3 \times 4)} \cdot

\begin{bmatrix}
\phi_0(X)\\
\phi_1(X)\\
\phi_2(X)\\
\phi_3(X)\\
\end{bmatrix}
$$

$$
x(X) = \vec{x_0} +

\underbrace{
\begin{bmatrix}
 \mid& \mid&  \mid& \mid  \\
 x_0&  x_1&  x_2&  x_3 \\
 \mid&  \mid&  \mid& \mid \\
\end{bmatrix}}_{(3 \times 4)} 

\underbrace{
\begin{bmatrix}
& & & -\vec{1}^{T} \; T^{-1} & & & \\
-&-&-&-&-&-& \\
& & & & & & \\
& & &T^{-1}& & & \\
& & & & & & \\
\end{bmatrix}}_{=D \; (4 \times 3)}

\underbrace{
(X - X_0)}_{(3 \times 1)}
$$

This equation is pretty useful as it allows to easily compute the deformation gradient tensor:

$$
F = \underbrace{\frac{\partial}{\partial X} x(X,t)}_{(3 \times 3)} = \underbrace{
\begin{bmatrix}
 \mid& \mid&  \mid& \mid  \\
 x_0&  x_1&  x_2&  x_3 \\
 \mid&  \mid&  \mid& \mid \\
\end{bmatrix}}_{(3 \times 4)} 

\underbrace{
\begin{bmatrix}
& & & -\vec{1}^{T} \; T^{-1} & & & \\
-&-&-&-&-&-& \\
& & & & & & \\
& & &T^{-1}& & & \\
& & & & & & \\
\end{bmatrix}}_{=D \; (4 \times 3)}


$$

---


<br>
<br>
<br>
<br>
<br>

---

## Kinetic energy

The kinetic energy of the system is given by:

$$
T = \sum_{i=1}^{N} \frac{1}{2} \int_{\Omega_{i,0}} \dot{q}(t, X)^T \, \dot{q}(t, X) \; d\Omega_{i,0}
$$

Where the sum runs over all the element cells of the mesh (tetrahedra in this case). $q(X,t)$ is the generalized coordinate of the system. In this case, the generalized coordinate is the displacement field $u(X,t)$. The dot denotes the time derivative. Note that the kinetic energy of the system is computed in the reference configuration (Lagrangian formulation).

For each element, we can write the displacement of a given point as: 

$$

u(X,t) = 

\underbrace{
\begin{bmatrix}
 & & & \\
\mathrm{I}\phi_0(X) & \mathrm{I}\phi_1(X) & \mathrm{I}\phi_2(X) & \mathrm{I}\phi_3(X)\\
 & & & \\
\end{bmatrix}}_{= N(X) \; (3 \times 12)}


\underbrace{
\begin{bmatrix}
\mid \\
u_0(t) \\
\mid \\
 \\
\mid\\
u_1(t) \\
\mid \\
... \\
\end{bmatrix}}_{=u(t) \; (1 \times 12)}
$$

The kinetic energy of the system is therefore given by:

$$
\sum_{i=1}^{N} \frac{1}{2} \dot{u}(t)^{T} \underbrace{\int_{\Omega_{i,0}}N(X)^T  N(X) \, d\Omega_{i,0}}_{= \mathrm{M}_0 (12 \times 12)} \; \dot{u}(t)
$$


Where $\mathrm{M}_0$ is the local mass matrix of the system: 

$$
\mathrm{M}_0 = 
\int_{\Omega_{i,0}}
\begin{bmatrix}
\mathrm{I} \phi_0(X) \phi_0(X) & \mathrm{I} \phi_0(X) \phi_1(X) & \mathrm{I} \phi_0(X) \phi_2(X) & \mathrm{I} \phi_0(X) \phi_3(X) \\
\mathrm{I} \phi_1(X) \phi_0(X) & \mathrm{I} \phi_1(X) \phi_1(X) & \mathrm{I} \phi_1(X) \phi_2(X) & \mathrm{I} \phi_1(X) \phi_3(X) \\
\mathrm{I} \phi_2(X) \phi_0(X) & \mathrm{I} \phi_2(X) \phi_1(X) & \mathrm{I} \phi_2(X) \phi_2(X) & \mathrm{I} \phi_2(X) \phi_3(X) \\
\mathrm{I} \phi_3(X) \phi_0(X) & \mathrm{I} \phi_3(X) \phi_1(X) & \mathrm{I} \phi_3(X) \phi_2(X) & \mathrm{I} \phi_3(X) \phi_3(X) \\
\end{bmatrix} d\Omega_{i,0}
$$

This local mass matrix can be computed like done in the previous notebooks by numerically integrating the shape functions in a reference element and 
then transforming the region of integration. The global matrix can also be assembled in a similar way as done in the previous notebooks.

---

<br>
<br>
<br>
<br>
<br>

---

## Potential energy: Strain Density Energy

The potential energy of the system in Lagrangian form is given by:

$$
V = \sum_{i=1}^{N} \frac{1}{2} \int_{\Omega_{i,0}} \Psi(\mathrm{E}) \; d\Omega_{i,0} - \int_{\Omega_{i,0}} f \cdot u \; d\Omega_{i,0} 
- \int_{\partial \Omega_{i,0}} t \cdot u \; d\partial \Omega_{i,0}
$$ 

$\Psi$ and $E$ are respectively the strain energy density function and the Green-Lagrange strain tensor of the system. $f$ is the body force acting on the system and $t$ is the traction force acting on the system. $u$ is the displacement field of the system.
$\partial \Omega_{i,0}$ denotes the boundary of the element cell $\Omega_{i,0}$.

For an isotropic linear elastic material, the strain energy density function in Lagrangian formulation is given by:

$$
\Psi(\epsilon) = \frac{1}{2} \mathrm{S} : \mathrm{E} 
$$ 

Where $\mathrm{S}$ is the second Piola-Kirchhoff stress tensor. And 
$:$ is the double contraction operation: $A:B = A_{1,1} B_{1,1} + A_{1,2} B_{1,2} + ... + A_{3,3} B_{3,3}$.


The second Piola-Kirchhoff stress tensor is related to the Cauchy stress tensor ($\sigma$) by:
$$
S = \frac{1}{J} \mathrm{F}^{-1} \sigma \mathrm{F}^{-T}
$$

Where $J$ is the determinant of the deformation gradient tensor $\mathrm{F}$.

In tensor form, S is for an isotropic linear elastic material given by:
$$
S = \lambda \mathrm{tr}(\mathrm{E}) \mathrm{I} + 2 \mu \mathrm{E}
$$

The 6 independent components of S can also be computed as a matrix vector product:
$$
\begin{bmatrix}
S_{11} \\
S_{22} \\
S_{33} \\
S_{12} \\
S_{23} \\
S_{13} \\
\end{bmatrix} =

\underbrace{
\begin{bmatrix}
\lambda + 2 \mu & \lambda & \lambda & 0 & 0 & 0 \\
\lambda & \lambda + 2 \mu & \lambda & 0 & 0 & 0 \\
\lambda & \lambda & \lambda + 2 \mu & 0 & 0 & 0 \\
0 & 0 & 0 & \mu & 0 & 0 \\
0 & 0 & 0 & 0 & \mu & 0 \\
0 & 0 & 0 & 0 & 0 & \mu \\
\end{bmatrix}}_{= \mathrm{C} \; (6 \times 6)}

\begin{bmatrix}
E_{11} \\
E_{22} \\
E_{33} \\
2 E_{12} \\
2 E_{23} \\
2 E_{13} \\
\end{bmatrix}
$$

Where $\lambda$ and $\mu$ (shear modulus) are the Lamé parameters of the material. $\mathrm{I}$ is the identity tensor and $\mathrm{tr}$ is the trace operator. Note that the first Lame parameters can be obtained from the Young's modulus and the Poisson's ratio of the material: 
$$
\lambda = \frac{E \nu}{(1+\nu)(1-2\nu)}
$$. 
$$
\mu = \frac{E}{2(1+\nu)} 
$$.

The strain density function in this case can therefore simply be written as:

$$
\Psi(\mathrm{E}) = \frac{1}{2} \mathrm{S} : \mathrm{E} = \frac{1}{2} \vec{\mathrm{E}}^T \mathrm{C} \vec{\mathrm{E}}
$$

In the limit of small strain: $\mathrm{E} \approx \frac{1}{2}(\nabla_X u + \nabla_X u^T)$, the element of the strain tensor can directly be obtained from the derivative of the displacement field:
$$
\begin{align}
E_{11} &= \frac{\partial u_1}{\partial X_1} \\
E_{22} &= \frac{\partial u_2}{\partial X_2} \\
E_{33} &= \frac{\partial u_3}{\partial X_3} \\
E_{12} &= \frac{1}{2} \left( \frac{\partial u_1}{\partial X_2} + \frac{\partial u_2}{\partial X_1} \right) \\
E_{23} &= \frac{1}{2} \left( \frac{\partial u_2}{\partial X_3} + \frac{\partial u_3}{\partial X_2} \right) \\
E_{13} &= \frac{1}{2} \left( \frac{\partial u_1}{\partial X_3} + \frac{\partial u_3}{\partial X_1} \right) \\
\end{align}
$$

This can be written in matrix form as:
$$
\begin{bmatrix}
E_{11} \\
E_{22} \\
E_{33} \\
2 E_{12} \\
2 E_{23} \\
2 E_{13} \\
\end{bmatrix} =

\underbrace{
\begin{bmatrix}
\frac{\partial}{\partial X_1} & 0 & 0 \\
0 & \frac{\partial}{\partial X_2} & 0 \\
0 & 0 & \frac{\partial}{\partial X_3} \\
\frac{\partial}{\partial X_2} & \frac{\partial}{\partial X_1} & 0 \\
0 & \frac{\partial}{\partial X_3} & \frac{\partial}{\partial X_2} \\
\frac{\partial}{\partial X_3} &  0 & \frac{\partial}{\partial X_1} \\
\end{bmatrix}}_{= B \; (6 \times 3)}
\;
u(X,t) = 
B \; 
\underbrace{
\begin{bmatrix}
I \phi_0(X_1, X_2, X_3) & I \phi_1(X_1, X_2, X_3) & I \phi_2(X_1, X_2, X_3) & I \phi_3(X_1, X_2, X_3) \\
\end{bmatrix}}_{=N(X) \; (3 \times 12)}  \;
\underbrace{
\begin{bmatrix}
\mid \\
u_0(t) \\
\mid \\
 \\
\mid\\
u_1(t) \\
\mid \\
... \\
\end{bmatrix}}_{=u(t) \; (12 \times 1)}

$$

$$
\begin{bmatrix}
E_{11} \\
E_{22} \\
E_{33} \\
2 E_{12} \\
2 E_{23} \\
2 E_{13} \\
\end{bmatrix} = B \; N(X) \; u(t) = 

\underbrace{
\begin{bmatrix}
\partial_{X_1} \phi_0& 0& 0& \partial_{X_1} \phi_1& 0& 0& \partial_{X_1} \phi_2& 0& 0& \partial_{X_1} \phi_3& 0& 0 \\
0& \partial_{X_2} \phi_0& 0& 0& \partial_{X_2} \phi_1& 0& 0& \partial_{X_2} \phi_2& 0& 0& \phi_3 \partial_{X_2}& 0 \\
0& 0& \partial_{X_3} \phi_0& 0& 0& \partial_{X_3} \phi_1& 0& 0& \partial_{X_3} \phi_2& 0& 0& \partial_{X_3} \phi_3 \\
\partial_{X_2} \phi_0& \partial_{X_1} \phi_0& 0& \partial_{X_2} \phi_1& \partial_{X_1} \phi_1& 0& \partial_{X_2} \phi_2& \partial_{X_1} \phi_2& 0& \phi_3 \partial_{X_2}& \partial_{X_1} \phi_3& 0 \\
0& \partial_{X_3} \phi_0& \partial_{X_2} \phi_0& 0& \partial_{X_3} \phi_1& \partial_{X_2} \phi_1& 0& \partial_{X_3} \phi_2& \partial_{X_2} \phi_2& 0& \partial_{X_3} \phi_3& \phi_3 \partial_{X_2} \\
\partial_{X_3} \phi_0& 0& \partial_{X_1} \phi_0& \partial_{X_3} \phi_1& 0& \partial_{X_1} \phi_1& \partial_{X_3} \phi_2& 0& \partial_{X_1} \phi_2& \partial_{X_3} \phi_3& 0& \partial_{X_1} \phi_3 \\
\end{bmatrix}}_{=\Gamma (6 \times 12)} \; u(t) 
$$

To create the $\Gamma$ matrix, it is pretty convenient to start by forming the following matrix:
$$
\begin{bmatrix}
\partial_{X_1} \phi_0& \partial_{X_2} \phi_0& \partial_{X_3} \phi_0 \\
\partial_{X_1} \phi_1& \partial_{X_2} \phi_1& \partial_{X_3} \phi_1 \\
\partial_{X_1} \phi_2& \partial_{X_2} \phi_2& \partial_{X_3} \phi_2 \\
\partial_{X_1} \phi_3& \partial_{X_2} \phi_3& \partial_{X_3} \phi_3 \\
\end{bmatrix} = 

\begin{bmatrix}
\phi_0(X_1, X_2, X_3) \\
\phi_1(X_1, X_2, X_3) \\
\phi_2(X_1, X_2, X_3) \\
\phi_3(X_1, X_2, X_3) \\
\end{bmatrix}
\cdot
\begin{bmatrix}
\partial_{X_1} & \partial_{X_2} & \partial_{X_3} \\
\end{bmatrix} =

\underbrace{
\begin{bmatrix}
& & & -\vec{1}^{T} \; T^{-1} & & & \\
-&-&-&-&-&-& \\
& & & & & & \\
& & &T^{-1}& & & \\
& & & & & & \\
\end{bmatrix}}_{=D \;(4 \times 3)}


\mathrm{I}
$$

Since the barycentric coordinate functions are linear with respect to the position in the reference configuration, their derivative are constant. Meaning that the Green-Lagrange tensor
and strain density energy are also constant in the elements. The integral of the strain density energy inside an element is therefore given by:
$$
\int_{\Omega_{i,0}} \Psi(\mathrm{E}) \; d\Omega_{i,0} = \text{Vol}(\Omega_{i,0}) \cdot \Psi(\mathrm{E}) = \text{Vol}(\Omega_{i,0}) \cdot \frac{1}{2} \vec{\mathrm{E}}^T \mathrm{C} \vec{\mathrm{E}} = 
\text{Vol}(\Omega_{i,0}) \cdot \frac{1}{2} u(t)^T \underbrace{\Gamma^T \mathrm{C} \Gamma}_{=K_0 \; (12 \times 12) } u(t)
$$

$K_0$ is the local stiffness matrix of the system. The global stiffness matrix can be assembled in a similar way as done in the previous notebooks.

---


<br>
<br>
<br>
<br>
<br>

---

## Potential energy: Body and Traction Forces



### Body forces

The body forces applied to an element of the mesh (gravity, etc.) will be given with respect to the deformed configuration:
$$
\int_{\Omega_{i}} \rho g^T \cdot u(x,t) \; d\Omega_{i}
$$

Where $\rho$ is the density of the material and $g$ is the acceleration due to gravity. Even though the gravity is constant throughout the element, 
the density between the deformed and reference configuration might not be the same. Because of the conservation of mass, we have that:
$$
\int_{\Omega_{i}} \rho g^T \cdot u(x,t) \; d\Omega_{i} = \int_{\Omega_{i,0}} \rho_0 g^T \cdot u(X,t) \; d\Omega_{i,0} 
$$

Since $u(X,t) = N(X) u(t)$, we can write the above equation as:
$$
\int_{\Omega_{i,0}} \rho_0 g^T \cdot u(X,t) \; d\Omega_{i,0} = \rho_0 g^T  \int_{\Omega_{i,0}} 
\underbrace{
\begin{bmatrix}
 & & & \\
\mathrm{I}\phi_0(X) & \mathrm{I}\phi_1(X) & \mathrm{I}\phi_2(X) & \mathrm{I}\phi_3(X)\\
 & & & \\
\end{bmatrix}}_{= N(X) \; (3 \times 12)}  \; d\Omega_{i,0}
\; \underbrace{
\begin{bmatrix}
\mid \\
u_0(t) \\
\mid \\
 \\
\mid\\
u_1(t) \\
\mid \\
... \\
\end{bmatrix}}_{=u(t) \; (12 \times 1)}
$$


Since all the calculation are run wrt the reference configuration, the inetgral in the above equation only has to be computed once at the beginning of the simulation. 


<br>
<br>

### Traction forces

The traction forces are given by the following surface integral:
$$
\int_{\partial \Omega_{i}} t^T u \; d\partial \Omega_{i}
$$

The traction forces are given with respect to the world/deformed configuration. Since, we are solving for the displacement with respect to the reference configuration, we need to transform the traction forces to the reference configuration. We first need to rotate the traction forces with respect to the axis orthogonal to the normals in the deformed and reference configuration. The rotation matrix is given by the Rodrigues formula:
$$
R = \mathrm{I} + \sin(\theta) \times \mathrm{K} + (1 - \cos(\theta)) \times \mathrm{K}^2
$$

With: 
$$
K = 
\begin{bmatrix}
0 & -k_z & k_y \\
k_z & 0 & -k_x \\
-k_y & k_x & 0 \\
\end{bmatrix}
$$ 

$ k = n \times N$ and $\theta = \arccos(n \cdot N)$, where $n$ is the unit normal in the deformed configuration and $N$ is the unit normal in the reference configuration. The traction forces then need to be rescaled based on the difference in area between the deformed ($a$) and reference ($A$) configuration. The traction forces in the reference configuration are then given by:
$$
\int_{\partial \Omega_{i}} t^T  u \; d\partial \Omega_{i} = \frac{a}{A} \int_{\partial \Omega_{i,0}} t^T R^T  u(X,t) \; d\partial \Omega_{i,0}
$$

Since the normals and traction vectors are constant accross the surface triangles, the integral can be rewritten as: 
$$
\frac{a}{A} \int_{\partial \Omega_{i,0}} t^T R^T  u(X,t) \; d\partial \Omega_{i,0} =  \frac{a}{A}  t^T R^T  \int_{\partial \Omega_{i,0}} \underbrace{
\begin{bmatrix}
 & & & \\
\mathrm{I}\phi_0(X) & \mathrm{I}\phi_1(X) & \mathrm{I}\phi_2(X) & \mathrm{I}\phi_3(X)\\
 & & & \\
\end{bmatrix}}_{= N(X) \; (3 \times 12)}
\; d\partial \Omega_{i,0}

\;

\underbrace{
\begin{bmatrix}
\mid \\
u_0(t) \\
\mid \\
 \\
\mid\\
u_1(t) \\
\mid \\
... \\
\end{bmatrix}}_{=u(t) \; (12 \times 1)}
$$









---


<br>
<br>
<br>
<br>
<br>

---

## Derivation of the equations of motion

We now need to solve the Euler-Lagrange equations to find the displacement field of the system. The Euler-Lagrange equations are given by:

$$
\frac{d}{dt} \left( \frac{\partial L}{\partial \dot{u}} \right) - \frac{\partial L}{\partial u} = 0
$$



---